In [78]:
import json
from pprint import pprint
import logging
import requests, json
from abc import ABC,  abstractclassmethod
from amocrm.v2 import tokens, Pipeline, Lead
import pandas as pd
from typing import Union, Optional, Tuple
import numpy as np 

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

logger = logging.getLogger(__name__)
logger.setLevel(logging.INFO)

In [3]:
with open('secrets.json', 'r') as f:
    secrets = json.load(f)

storage = tokens.FileTokensStorage()
tokens.default_token_manager(
    client_id=secrets['client_id'],
    client_secret=secrets['client_secret'],
    subdomain=secrets['subdomain'],
    redirect_url=secrets['redirect_url'],
    storage=storage
)
access_token = tokens.default_token_manager.get_access_token()

with open('events.json', 'r',encoding='utf-8') as f:
    EVENT_DIRECTORY = json.load(f)
    

In [166]:
class SpecificDataProcessing:
    """
    Обработка specific_data в зависимоси от типа входящей строки
    """
    
    def __init__(self):
        """
        entity_linked_func - получить даные о клиенте/компании
        sale_field_changed_func- поулчить данные о цене 
        lead_status_func - получить данные о статусе задачи 
        pipline_func - получить даные о pipline 
        
        sale: float - цена 
        company: int - id компании 
        client: int - id клиента
        lead_status: int - id статуса задачи 
        pipline: int - id пайплайна
        """
        
        # логика обработки для разных типов specific_data
        self.entity_linked_func = lambda row: row.specific_data['after'][0]['link']['entity']['id']                
        self.sale_field_changed_func = lambda row: row.specific_data['after'][0]['sale_field_value']['sale']
        self.lead_status_func = lambda row: row.specific_data['after'][0]['lead_status']['id']
        self.pipline_func = lambda row: row.specific_data['after'][0]['lead_status']['pipeline_id'] 
        
        # начальная инициализация сквозных значений (sale, responsible_user_id, pipeline, lead_status)
        self.initial_func= lambda row: (row.specific_data['sale'], 
                                        row.specific_data['responsible_user_id'],
                                        row.specific_data['pipeline'],
                                        row.specific_data['lead_status'])
        
        # сквозные поля датасета
        self.sale: Optional[float] = np.nan
        self.company: Optional[str]= np.nan 
        self.contact: Optional[str]= np.nan
        self.lead_status: Optional[str] = np.nan
        self.pipline: Optional[int]= np.nan 
        self.responsible: Optional[int]= np.nan 
    
    def __call__(self, row: pd.Series) -> pd.Series:
        """  
        Обработка поля specific_data для получени сквозных показателей
        
        Аргументы:
            row: строка данных 
        Возвращает:
            pd.Series для следующих полей:
                ('client', 'company', 'sale', 'lead_status', 'pipline')
        """
        
        # если это строка инициализации задачи
        if row.type == 'initial':
            self.sale, self.responsible, self.pipline, self.lead_status = self.initial_func(row)
        
        # если установили/изменили sale
        elif row.type == 'sale_field_changed':
           self.sale =  self.sale_field_changed_func(row)
           
        # если добавили/изменили сущности: company, contact
        elif row.type=='entity_linked':
            if row.specific_data['after'][0]['link']['entity']['type']=='contact':
                 self.contact = self.entity_linked_func(row)
            elif row.specific_data['after'][0]['link']['entity']['type']=='company':
                self.company = self.entity_linked_func(row)
                
        # если изменился статус/пайплайн задачи 
        elif row.type=='lead_status_changed':
            self.lead_status = self.lead_status_func(row)
            self.pipline = self.pipline_func(row)

        return pd.Series([self.contact, self.company, self.sale, self.lead_status, self.pipline, self.responsible])

In [170]:
class AmoCRM:
    def __init__(self, token_manager, secrets):
        """ 
        Получение и обработка джанных AmoCRM
        
        Аргументы:
            token_manager - получение и обновление токена для доступа к данным 
            subdomain - поддомен аккаунта в autocrm 
            processor - класс для обработки сквозных значений
            _general_fields - поля основных значений (одинаковые для каждого типа записи)
        """
        
        self.token_manager = token_manager
        self.subdomain = secrets["subdomain"]
        self.processor = SpecificDataProcessing
        self._general_fields = ['type', 'entity_id', 'created_by', 'created_at', 'specific_data']

    def _api_call(self, endpoint, entity, entity_id):
        headers = {
            'Authorization': f'Bearer {self.token_manager.get_access_token()}',
            'Content-Type': 'application/json'
        }

        url = f'https://{self.subdomain}.amocrm.ru/api/v4/{endpoint}'

        params = {
            "filter[entity]": entity,
            "filter[entity_id]": entity_id
        }

        response = requests.get(url, headers=headers, params=params)
        response.raise_for_status()
        return response
    

    def _initial_processing(self, initial_df: pd.DataFrame)-> pd.DataFrame:
        ''' 
        Унификация инициализирующих данных
        '''

        initial_df['type']='initial',
        initial_df['entity_id']=initial_df['id']
        initial_df['specific_data'] = initial_df.apply(
            lambda row: {'sale': row.price, 
                        'responsible_user_id': row.responsible_user_id, 
                        'pipeline': row.pipeline_id, 
                        'lead_status': row.status_id}, axis=1)
        
        return initial_df[self._general_fields]
        
    
    def _event_processing(self, event_df: pd.DataFrame) -> pd.DataFrame:
        '''
        Унификация данных по евентам
        '''
        
        event_df['specific_data'] = event_df.apply(
            lambda row: {'after':row.value_after, 'before':row.value_before}, axis=1)
        
        return event_df[self._general_fields]
    
    def _task_processing(self, task_df: pd.DataFrame) -> pd.DataFrame:
        '''
        Унификация данных по задачам
        '''
        
        task_df['type'] = 'task'
        task_df['specific_data'] = task_df.apply(
            lambda row: {'text': row.text, 
                         'is_completed': row.is_completed,
                         'result':row.result, 
                         'responsible_user_id': row.responsible_user_id,
                         'complete_till': row.complete_till}, axis=1)
        
        return  task_df[self._general_fields ]


    def _note_processing(self, note_df: pd.DataFrame) -> pd.DataFrame:
        '''
        Унификация данных по заметкам
        '''
        
        note_df['type'] = 'note' 
        note_df['specific_data'] = note_df.apply(
            lambda row: {'text': row.params['text'],
                         'note_type': row.note_type,
                         'responsible_user_id': row.responsible_user_id,
                         'updated_at': row.updated_at}, axis=1)
        
        return note_df[self._general_fields]

    def get_initial_data_lead(self, lead_id):
        """ 
        Получение начальных данных по задаче
        """
        
        headers = {
        'Authorization': f'Bearer {self.token_manager.get_access_token()}',
        'Content-Type': 'application/json'
        }
        url = f'https://{secrets["subdomain"]}.amocrm.ru/api/v4/leads/{lead_id}'
        response = requests.get(url, headers=headers)
        
        if response.status_code == 200:
            initial_df = pd.Series(response.json()).to_frame().T # 
            return  self._initial_processing(initial_df)
        
        elif response.status_code == 204:
            return pd.DataFrame()
        
        else:
            raise Exception('Error: {}'.format(response.status_code))
        

    def get_events_by_lead_id(self, lead_id):
        response = self._api_call('events', 'lead', lead_id)
        if response.status_code == 200:
            event_df_raw = pd.DataFrame(response.json()['_embedded']['events'])
            return self._event_processing(event_df_raw)
        
        elif response.status_code == 204:
            return pd.DataFrame()
        
        else:
            raise Exception('Error: {}'.format(response.status_code))


    def get_tasks_by_lead_id(self, lead_id):
        response = self._api_call('tasks', 'lead', lead_id)
        if response.status_code == 200:
            task_df_raw = pd.DataFrame(response.json()['_embedded']['tasks'])
            return self._task_processing(task_df_raw)
        
        elif response.status_code == 204:
            return pd.DataFrame()
        
        else:
            raise Exception('Error: {}'.format(response.status_code))


    def get_notes_by_lead_id(self, lead_id):
        response = self._api_call('leads/notes', 'lead', lead_id)
        if response.status_code == 200:
            note_df_raw = pd.DataFrame(response.json()['_embedded']['notes'])
            return self._note_processing(note_df_raw)
        
        elif response.status_code == 204:
            return pd.DataFrame()
        
        else:
            raise Exception('Error: {}'.format(response.status_code))
    
    # preparation all_lead_info
    def get_all_lead_info(self, lead_id):
        """
        Получение и обоработка все данных по задачам
        """
        
        processor = self.processor() # обработчик сквозных значений
        inital_df = crm.get_initial_data_lead(lead_id)
        events_df = self.get_events_by_lead_id(lead_id)
        tasks_df = self.get_tasks_by_lead_id(lead_id)
        notes_df = self.get_notes_by_lead_id(lead_id)
        
        result = pd.concat([inital_df, events_df, tasks_df, notes_df], axis=0).\
                sort_values('created_at').reset_index(drop=True).assign(client=None,
                                                                        company=None,
                                                                        sale=None,  
                                                                        lead_status=None,
                                                                        pipline=None,
                                                                        responsible=None)
                
        result[['client', 'company', 'sale', 'lead_status', 'pipline', 'responsible']] = result.apply(processor , axis=1)
        
        return result.astype({'client': pd.Int32Dtype(), 
                              'company': pd.Int32Dtype(), 
                              'sale': pd.Float32Dtype(),
                              'lead_status': pd.Int32Dtype(),
                              'pipline':pd.Int32Dtype(),
                              'responsible':pd.Int32Dtype()})
# example
# lead_id = 37925381 
#lead_id = 37903589
lead_id =37903589
crm = AmoCRM(tokens.default_token_manager, secrets)
crm.get_all_lead_info(lead_id)

,type,entity_id,created_by,created_at,specific_data,client,company,sale,lead_status,pipline,responsible
0,initial,37903589,0,1694673386,"{'sale': 44041, 'responsible_user_id': 10076546, 'pipeline': 7236182, 'lead_status': 60363374}",<NA>,<NA>,44041.0,60363374,7236182,10076546
1,sale_field_changed,37903589,0,1694673386,"{'after': [{'sale_field_value': {'sale': 44041}}], 'before': []}",<NA>,<NA>,44041.0,60363374,7236182,10076546
2,name_field_changed,37903589,0,1694673386,"{'after': [{'name_field_value': {'name': 'Сделка test_2'}}], 'before': []}",<NA>,<NA>,44041.0,60363374,7236182,10076546
3,lead_added,37903589,0,1694673386,"{'after': [], 'before': []}",<NA>,<NA>,44041.0,60363374,7236182,10076546
4,entity_linked,37903589,10076546,1696576254,"{'after': [{'link': {'entity': {'type': 'company', 'id': 71291517}}}], 'before': []}",<NA>,71291517,44041.0,60363374,7236182,10076546
5,entity_linked,37903589,10076546,1696576255,"{'after': [{'link': {'entity': {'type': 'contact', 'id': 71291521}}}], 'before': []}",71291521,71291517,44041.0,60363374,7236182,10076546
